### Selenium Scraping

#### 1. Gmarket 베스트 셀러 데이터 스크래핑 
- 전체 200개중 상위 10개 출력
- pandas 데이터를 분석할수 있는 python 라이브러리

    - pip3 install pandas
    - 데이터 프레임 생성 : df = pd.DataFrame(columns=["A","B"])
    - 데이터 추가 : df.loc[len(df)] = [{"A":1,"B":2}, {"A":3,"B":4}]

In [3]:
from selenium import webdriver
import pandas as pd

In [4]:
def get_gmarket_bestsellers(number=10):
    
    def to_int(price):
        return int(price.replace(",","").replace("원","")) # 숫자로 만들기 위해 가격에서 콤마(,)와 '원'을 삭제
    
    driver =  webdriver.Chrome()
    driver.get('http://corners.gmarket.co.kr/Bestsellers')
    
    bestlist_selector = driver.find_elements_by_css_selector("#gBestWrap .best-list")[1]

    items = bestlist_selector.find_elements_by_css_selector("li")[:number]
    
    df = pd.DataFrame(columns=["상품명","원가","판매가","이미지","상품링크"])
    
    for item in items:

        idx = item.find_element_by_css_selector("p").text # 순번
        img_link = item.find_element_by_css_selector("img").get_attribute("src") # 이미지 URL
        item_link = item.find_element_by_css_selector(".itemname").get_attribute("href") # 아이템 URL
        content = item.find_element_by_css_selector(".itemname").text # 아이템 텍스트
        o_price = item.find_element_by_css_selector(".item_price .o-price").text # 오리지날 가격
        s_price = item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[0] # 판매 가격
        o_price = o_price if o_price else s_price # 원가 데이터가 없는경우 판매가를 대입
        
        data = {"상품명": content,
                "원가": to_int(o_price), 
                "판매가": to_int(s_price),
                "이미지": img_link, 
                "상품링크": item_link, }
        
        # Pandas 데이터 입력
        df.loc[len(df)] = data
        
    driver.close()
    return df

In [5]:
%%time
result = get_gmarket_bestsellers(200)

CPU times: user 1.84 s, sys: 91.7 ms, total: 1.93 s
Wall time: 31.8 s


In [6]:
result.head() # 상위 6개 보여주기

,상품명,원가,판매가,이미지,상품링크
0,[시크릿데이](최종가 8900원) 러브 생리대 9팩+세안밴드or다이어리,21900,10900,http://gdimg.gmarket.co.kr/291472256/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
1,[서울우유]서울우유 나무3종150mlx24입 리이브사과/포도/감귤,10000,4900,http://gdimg.gmarket.co.kr/464325238/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
2,[앤디애플]봄오픈/아동복/티셔츠/원피스/레깅스/신학기/상하복,26000,7900,http://gdimg.gmarket.co.kr/301110076/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
3,봄신상 배기바지2900원부터 티셔츠 맨투맨,29000,8900,http://gdimg.gmarket.co.kr/166866548/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
4,[요즘에]20%쿠폰/신상특가/청바지/슬랙스/빅사이즈,33000,10000,http://gdimg.gmarket.co.kr/189842449/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...


In [7]:
# Pandas로 할인율 계산
result["할인율"] = 100 - (result["판매가"] / result["원가"]) * 100
result["할인율"] = result["할인율"].astype(dtype="float64")
result["할인율"] = round(result["할인율"], 2)
result = result.sort_values("할인율", ascending=False).reset_index().rename(columns={"index": "rank"}) # 할인율이 높은 순서대로 랭크
result.head()

,rank,상품명,원가,판매가,이미지,상품링크,할인율
0,117,[쁘리엘르]100% 순면차렵이불/패드세트/마이크로화이바/피치스킨,99900,19900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,80.08
1,199,[릴리푸리]여아의류/봄신상/릴리푸리/원피스/레깅스/치렝스,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
2,169,[팬콧키즈]팬콧 도티잠뜰 신학기 봄 실내화 슬리퍼,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
3,34,이사도라 사랑스러운 그녀 플랫/로퍼/단화,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
4,46,[쿠미다]핸드폰케이스 1+1 갤럭시노트8 S8 S7 아이폰8 X G6 외,13000,3900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00


In [6]:
# utf-8로 인코딩한 csv 파일로 저장
result.to_csv("gmarket.csv", encoding="utf-8")

In [7]:
pd.read_csv("gmarket.csv").head() # 저장 잘 되었는지 확인

,Unnamed: 0,rank,상품명,원가,판매가,이미지,상품링크,할인율
0,0,194,[다샵]벽걸이 스탠드 에어컨커버 제습기/선풍기커버,26900,6500,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,75.84
1,1,139,[아망떼]순면100%모닝벨 매트리스커버/이불커버/침대패드,33000,8900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,73.03
2,2,165,쁘띠뮤/신학기 아동정장 원피스 레깅스 트레이닝,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
3,3,59,[젤리스푼]아동복/봄신상오픈/상하복/티셔츠/레깅스/팬츠/점퍼,43000,12900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
4,4,121,[디즈니]디즈니 아동 캐릭터 실내화 슬리퍼 학생 덧신 샌들,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00


#### 2. Naver 메일 데이터 스크래핑

- 네이버 메일에 로그인한후 데이터 가져오기 (로그인을 해야 볼 수 있는)

In [8]:
from selenium import webdriver
import pickle
import time

In [17]:
# 패스워드 저장하기
pw = ""
file = open("pw.pickle", "wb")
pickle.dump(pw, file)

In [10]:
main_url = "http://naver.com"
mail_url = "http://mail.naver.com"

In [15]:
# 브라우져 오픈 및 메인 사이트 이동
driver = webdriver.Chrome()
driver.get(main_url)

In [18]:
# 아이디 패스워드 입력
pw = pickle.load(open("pw.pickle", "rb")) # pw 변수에 본인의 패스워드 입력
driver.find_element_by_css_selector( "#id" ).send_keys( "jsi2944" )
driver.find_element_by_css_selector( "#pw" ).send_keys( pw )

In [19]:
# 로그인 버튼 클릭
driver.find_element_by_css_selector("span.btn_login input[type=submit]").click()

In [20]:
# 메일 페이지로 이동
driver.get(mail_url)

In [21]:
# 메일 갯수
total_mail = driver.find_element_by_css_selector("#headTotalNum").text
total_mail = int(total_mail)
total_mail

159

In [22]:
# 안읽은 메일 갯수
not_read_mail = driver.find_element_by_css_selector("#headUnreadNum").text
not_read_mail = int(not_read_mail)
not_read_mail

157

In [23]:
# 메일 제목, 보낸사람 이름 가져오기
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  100


,제목,보낸사람,메일링크
95,(광고) 2월 20일 지성인님께 <치아교정 지원자> 모집 안내 드립니다.,지원자모집안내,https://mail.naver.com/read/popup/?nMailId=73008
96,[토익&토익Speaking] 토익 990점을 만드는 하루 10분 학습!,YBM인강 학습메일,https://mail.naver.com/read/popup/?nMailId=73007
97,(광고) 월간 주방용품! 요즘 트렌드 구경해볼까요?,GS SHOP,https://mail.naver.com/read/popup/?nMailId=73006
98,(광고) (얼굴 공개 X) 치아교정 지원자를 찾습니다.,지원자모집안내,https://mail.naver.com/read/popup/?nMailId=73005
99,"(광고)지*인님!봄!! 데일리 청 자켓 50%할인 19,800원..품절주의!!",ganzinara,https://mail.naver.com/read/popup/?nMailId=73004


In [24]:
# marked 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li.mark")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  0


,제목,보낸사람,메일링크


In [25]:
# 읽은 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li:not(.notRead)")
print("total mails: ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

total mails:  2


,제목,보낸사람,메일링크
0,[기본훈련/기본교육]예비군 훈련소집통지서 입니다.(0308 기본훈련 4기),51사단 168연대 3대대 ..,https://mail.naver.com/read/popup/?nMailId=73081
1,[기본훈련]예비군 훈련 안내문,51사단 168연대 3대대 ..,https://mail.naver.com/read/popup/?nMailId=73071


In [26]:
# 로그아웃
logout_url = driver.find_element_by_css_selector(".gnb_btn_login").get_attribute("href")
driver.get(logout_url)

In [27]:
# 브라우져 닫기
driver.close()